In [1]:
import os
import shutil

# import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class HuggingFaceEmbeddings(Embeddings):
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2'):
        # Load your Hugging Face model
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        """
        Embeds a list of documents.
        Args:
            texts (List[str]): List of document strings.
        Returns:
            List[List[float]]: List of embeddings for each document.
        """
        return self.model.encode(texts, convert_to_tensor=False).tolist()

    def embed_query(self, text):
        """
        Embeds a single query.
        Args:
            text (str): Query string.
        Returns:
            List[float]: Embedding for the query.
        """
        return self.model.encode([text], convert_to_tensor=False)[0].tolist()

embedding = HuggingFaceEmbeddings()

c:\Users\Hari\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# API + LLM Setup

os.environ["GOOGLE_API_KEY"] = ""

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(State)
memory = MemorySaver()

graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}

In [4]:
CHROMA_PATH = "chroma"
db = Chroma(
    persist_directory=CHROMA_PATH, embedding_function=embedding
)

C:\Users\Hari\AppData\Local\Temp\ipykernel_58596\2460391798.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [5]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [6]:
# Example conversation loop
print("Start conversing with the Gemini LLM... (Type 'exit' to stop)")

while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        print("Ending conversation. Goodbye!")
        break
    results = db.similarity_search_with_score(user_input, k=5)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _ in results])

    messages = PROMPT_TEMPLATE.format(context=context_text, question=user_input)
    config = {"configurable": {"thread_id": "1"}}
    
    events = graph.stream({"messages": [("user", messages)]}, config, stream_mode="values")

    for event in events:
        event["messages"][-1].content
    
    response_text = event["messages"][-1].content

    print(f"Bot: {response_text}")

Start conversing with the Gemini LLM... (Type 'exit' to stop)


c:\Users\Hari\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Bot: The provided text states that Mensa Brands has raised approximately $50 million in a Series A round.  Another source mentions a further $33 million raised in August, and a total of over $300 million in funding.

Bot: Mensa Brands acquires digital-first brands, primarily in fashion, beauty, cosmetics, and home furnishing.  They then help these brands scale through tech-led marketing, sales channel optimization, supply chain expertise, inventory management, and product innovation.  They aim to grow these brands both domestically and internationally.  Mensa keeps the founders involved, gradually acquiring full ownership over five years.

Ending conversation. Goodbye!


In [ ]:
# Exploring the i/o history
snapshot = graph.get_state(config)
for i in snapshot.values['messages']:
    print(i.content)


Answer the question based only on the following context:

Source: YourStory
Mensa Brands on Monday announced the closure of its Series A funding round, raising approximately $50 million. The round was led by Accel Partners, Falcon Edge
Capital, Norwest V enture Partners as well as prominent angel investors such as Kunal Shah, Mukesh Bansal, Rahul Mehta of  DST  Global, and Scott Shleifer of Tiger Global.
Additionally , debt financing facilities from Alteria Capital and InnoV en Capital are also secured, the company stated.  
According to the startup, the fund will be used to nurture, accelerate and scale digital-first brands. 
Speaking on the development, Ananth Narayanan, Founder and CEO of Mensa Brands, said" “Scaling digital brands from India is a large opportunity . Incredibly excited and
passionate about partnering with terrific founders and helping scale their brands globally . Having seen this first hand at Myntra and Medlife, we know the ef fort it takes to scale
a brand digit